In [1]:
import pandas as pd
import numpy as np

In [2]:
# def check_same_quartier(df1, df2, col_nbquartier_df1, col_nbquartier_df2, col_namequartier_df1, col_namequartier_df2):
def check_same_quartier(df1, df2):
    for row in df1.itertuples():
        curr_no_qr = row.no_qr
        curr_nom_qr = row.nom_qr
        is_same = curr_nom_qr == df2[df2['no_qr'] == curr_no_qr]['nom_qr']
        if is_same.bool()==False:
            print(curr_no_qr, curr_nom_qr, df2[df2['no_qr'] == curr_no_qr]['nom_qr'])

# check_same_quartier(df_nb_logements, df_quartier)

In [12]:
# Importation des quartiers
df_quartier = pd.read_csv('quartierreferencehabitation.csv')
df_quartier.sort_values(by=['no_qr'], inplace=True)

# Importations du nombre de logements par quartier
df_nb_logements = pd.read_csv('nombrelogementsquartiersreference.csv')
df_nb_logements.rename(columns={'No_QR': 'no_qr', 'Nom_QR': 'nom_qr'}, inplace=True)
df_nb_logements.sort_values(by=['no_qr'], inplace=True)

# Importation du nombre de logements sociaux par quartier
df_nb_log_sociaux = pd.read_csv('logements-sociaux.csv', sep=';', encoding='latin-1')
df_nb_log_sociaux.rename(columns={'qr': 'no_qr'}, inplace=True)

final_df = pd.DataFrame()

df_quartier[df_quartier['nom_mun'] == 'Montréal']
final_df['no_qr'] = df_quartier[df_quartier['nom_mun'] == 'Montréal']['no_qr']
final_df['nom_qr'] = df_quartier[df_quartier['nom_mun'] == 'Montréal']['nom_qr']

In [13]:
# Ajout du nombre d'habitants
nb_logements = []
for curr_no_qr in final_df['no_qr']:
    a = df_nb_logements[df_nb_logements['no_qr'] == curr_no_qr]['Nb_log'].tolist()
    nb_logements.append(a[0])
final_df['nb_logements'] = nb_logements

# Ajout du nombre de logements sociaux
nb_log_sociaux = []
for curr_no_qr in final_df['no_qr']:
    a = df_nb_log_sociaux[df_nb_log_sociaux['no_qr'] == curr_no_qr]['nb_log'].tolist()
    nb_log_sociaux.append(sum(a))
final_df['nb_log_sociaux'] = nb_log_sociaux

# Ajout proportion logements sociaux
a = final_df['nb_logements'].tolist()
b = final_df['nb_log_sociaux'].tolist()
prop_log_sociaux = np.divide(np.array(b), np.array(a)) * 100
final_df['prop_log_sociaux'] = prop_log_sociaux

In [15]:
final_df

,no_qr,nom_qr,nb_logements,nb_log_sociaux,prop_log_sociaux
52,01,Cartierville,10302,1003,9.735974
51,02,Nouveau-Bordeaux,12597,466,3.699293
13,03,Nicolas-Viel,9092,285,3.134624
11,04,Sault-au-Récollet,13129,742,5.651611
12,05,Saint-Sulpice,9793,1133,11.569488
...,...,...,...,...,...
71,71,Pierrefonds-Ouest,11068,82,0.740875
86,72,Pierrefonds-Est,15234,1302,8.546672
53,73,Anjou,20127,403,2.002285
54,74,Outremont,10309,93,0.902124
